# Biblioteka Bokeh

Bokeh to biblioteka wyróżniająca się wysokim poziomem interaktywności. Umożliwia dostosowywanie wizualizacji w czasie rzeczywistym  dla użytkowników, którzy nie mają styczności z kodem.

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show
# plus inne dodatkowe potrzebne do przygotowywanych wizualizacji

In [2]:
import numpy as np
import pandas as pd

#import matplotlib.pyplot as plt
#import seaborn as sns

data = np.genfromtxt('athlete_events_np.csv',
                     delimiter=';',
                     dtype=[
                         ('ID', 'i4'),
                         ('Name', 'U100'),
                         ('Sex', 'U1'),
                         ('Age', 'f4'),
                         ('Height', 'f4'),
                         ('Weight', 'f4'),
                         ('Team', 'U100'),
                         ('NOC', 'U3'),
                         ('Games', 'U100'),
                         ('Year', 'i4'),
                         ('Season', 'U100'),
                         ('City', 'U100'),
                         ('Sport', 'U100'),
                         ('Event', 'U100'),
                         ('Medal', 'U100')
                     ], names=True)
df = pd.DataFrame(data)

--------------------

##### ⭐ Zadanie 1: 

Przygotuj wykres punktowy (`circle`) opierając się na danych, które sam wybierzesz w sensowny sposób. W rozwiązaniu zdefiniuj własny pasek narzędzi składający się z narzędzi:

- `LassoSelectTool` i `xPanTool` z kategorii narzędzi *Gestures* (*Pan/Drag tools*),
- `PolySelectTool` z kategorii narzędzi *Gestures* (*Click/Tap tools*),
- `WheelZoomTool` z kategorii narzędzi *Gestures* (*Scroll/Pinch tools*),
- `UndoTool`, `RedoTool`, `ResetTool` i `SaveTool` z kategorii narzędzi *Actions*,
- `HoverTool` z etykietami danych z kategorii narzędzi *Inspectors*.

Zapoznaj się z parametrem `toolbar_location` i dobierz dla niego nową wartość. Zapoznaj się z metodą `autohide` obiektu `toolbar` i dobierz dla niego taką wartość, żeby pasek narzędzi chował się po zjechaniu kursorem myszy z wizualizacji. Zadbaj o czytelność wykresu (tytuł wykresu i podpisy osi). 

In [60]:
from bokeh.layouts import column
from bokeh.models import LassoSelectTool, PanTool, PolySelectTool, WheelZoomTool, UndoTool, RedoTool, \
    ResetTool, SaveTool, HoverTool

height = (data['Height'])[
    (data['NOC'] == 'POL') & (data['Year'] >= 2012) & (~np.isnan(data['Height'])) & (~np.isnan(data['Weight']))]
weight = (data['Weight'])[
    (data['NOC'] == 'POL') & (data['Year'] >= 2012) & (~np.isnan(data['Height'])) & (~np.isnan(data['Weight']))]

p = figure(title='Weight vs Height for Polish athletes since 2012', x_axis_label='Height', y_axis_label='Weight',
           tools=[LassoSelectTool(), PanTool(), PolySelectTool(), WheelZoomTool(), UndoTool(), RedoTool(), ResetTool(),
                  SaveTool(), HoverTool(tooltips=[('Height', '@x'), ('Weight', '@y')])],
           toolbar_location='above')
p.circle(x=height, y=weight, radius=0.5, color='navy', alpha=0.5)
p.toolbar.autohide = True

show(column(p))

##### ⭐ Zadanie 2:

Przygotuj wykres liniowy (`line`) opierając się na danych, które sam wybierzesz w sensowny sposób. Uwzględnij co najmniej 3 serie danych. Każda seria danych musi być przedstawiona na osobnym podwykresie (`subplot`) jednego obrazu. Zapoznaj się z metodami `column`, `row` i `gridplot` oraz dobierz dla nich właściwą wartość mając na uwadze wizualizację określonej liczby serii danych na osobnych podwykresach. Dodaj powiązane zachowania pomiędzy podwykresami:

- współdzielony ruchomy zakres dla wartości na osi X, także przy poziomym przesuwaniu wykresu,
- współdzielony wskaźnik będący narzędziem `CrosshairTool` widoczny jednocześnie na wszystkich podwykresach.

Zadbaj o czytelność wykresu (tytuł wykresu, podpisy osi i ewentualnie legenda). 

In [65]:
from bokeh.layouts import row
from bokeh.models import CrosshairTool, Span
avg_age_per_year_usa = df[df.NOC == 'USA'].groupby('Year').mean(numeric_only=True)[['Age']]
avg_age_per_year_chn = df[df.NOC == 'CHN'].groupby('Year').mean(numeric_only=True)[['Age']]
avg_age_per_year_pol = df[df.NOC == 'POL'].groupby('Year').mean(numeric_only=True)[['Age']]

for year in list(dict.fromkeys((list(avg_age_per_year_usa.index) + list(avg_age_per_year_chn.index) + list(avg_age_per_year_pol.index)))):
    if year not in avg_age_per_year_usa.index:
        avg_age_per_year_usa.loc[year] = 0
    if year not in avg_age_per_year_chn.index:
        avg_age_per_year_chn.loc[year] = 0
    if year not in avg_age_per_year_pol.index:
        avg_age_per_year_pol.loc[year] = 0

avg_age_per_year_usa.sort_index(inplace=True)
avg_age_per_year_chn.sort_index(inplace=True)
avg_age_per_year_pol.sort_index(inplace=True)

width = Span(dimension="width", line_width=1)
height = Span(dimension="height", line_width=1)

crosshair = CrosshairTool(overlay=[width, height])

p1 = figure(title="USA athletes average age per year", x_axis_label='Year', y_axis_label='Age', y_range=(0, 40))
p1.line(avg_age_per_year_usa.index, avg_age_per_year_usa, line_width=2)
p1.add_tools(crosshair)

p2 = figure(title="CHN athletes average age per year", x_axis_label='Year', y_axis_label='Age', y_range=(0, 40))
p2.line(avg_age_per_year_chn.index, avg_age_per_year_chn, line_width=2)
p2.add_tools(crosshair)

p3 = figure(title="POL athletes average age per year", x_axis_label='Year', y_axis_label='Age', y_range=(0, 40))
p3.line(avg_age_per_year_pol.index, avg_age_per_year_pol, line_width=2)
p3.add_tools(crosshair)

# Tworzenie siatki z podwykresami

# show(gridplot(children=[p1, p2, p3], ncols=2))
# show(column(p1, p2, p3))
show(row(p1, p2, p3))

##### ⭐ Zadanie 3:

Przygotuj wykres liniowy (`multi_line`) opierając się na danych, które sam wybierzesz w sensowny sposób. Uwzględnij co najmniej 3 serie danych. Zapoznaj się z parametrem `location` obiektu `legend` i dobierz dla niego nową wartość. Zapoznaj się z parametrem `click_policy` obiektu `legend` i dobierz dla niego taką wartość, żeby po kliknięciu w tytuł danej serii danych w legendzie, jej wizualizacji znikała z wykresu. Zadbaj o czytelność wykresu (tytuł wykresu, podpisy osi i legenda). 

In [23]:
avg_age_per_year_usa = df[df.NOC == 'USA'].groupby('Year').mean(numeric_only=True)[['Age']]
avg_age_per_year_chn = df[df.NOC == 'CHN'].groupby('Year').mean(numeric_only=True)[['Age']]
avg_age_per_year_pol = df[df.NOC == 'POL'].groupby('Year').mean(numeric_only=True)[['Age']]

for year in list(dict.fromkeys((list(avg_age_per_year_usa.index) + list(avg_age_per_year_chn.index) + list(avg_age_per_year_pol.index)))):
    if year not in avg_age_per_year_usa.index:
        avg_age_per_year_usa.loc[year] = pd.Series(dtype='float64')
    if year not in avg_age_per_year_chn.index:
        avg_age_per_year_chn.loc[year] = pd.Series(dtype='float64')
    if year not in avg_age_per_year_pol.index:
        avg_age_per_year_pol.loc[year] = pd.Series(dtype='float64')

avg_age_per_year_usa.sort_index(inplace=True)
avg_age_per_year_chn.sort_index(inplace=True)
avg_age_per_year_pol.sort_index(inplace=True)

p = figure(title='Athletes average age per year', x_axis_label='Year', y_axis_label='Age')

p.line(avg_age_per_year_usa.index, avg_age_per_year_usa.Age, line_color='blue', line_width=2, legend_label='USA')
p.line(avg_age_per_year_chn.index, avg_age_per_year_chn.Age, line_color='red', line_width=2, legend_label='CHN')
p.line(avg_age_per_year_pol.index, avg_age_per_year_pol.Age, line_color='green', line_width=2, legend_label='POL')

p.legend.click_policy="hide"
p.legend.location = "top_left"

show(p)

##### ⭐ Zadanie 4:

Przedstaw na dowolnym wykresie dowolne zestawienie danych przygotowane na podstawie pliku `athlete_events.csv` z pierwszego tygodnia. Zadbaj o czytelność wykresu. Do swojej wizualizacji dodaj co najmniej 3 różne widgety, które będą miały na nią wpływ:

- `AutocompleteInput`,
- `Button`,
- `CheckboxButtonGroup`,
- `CheckboxGroup`,
- `ColorPicker`,
- `DataCube`,
- `DataTable`,
- `DatePicker`,
- `DateRangePicker`,
- `MultipleDatePicker`,
- `DatetimePicker`,
- `DatetimeRangePicker`,
- `MultipleDatetimePicker`,
- `TimePicker`,
- `DateRangeSlider`,
- `DateSlider`,
- `DatetimeRangeSlider`,
- `Div`,
- `Dropdown`,
- `FileInput`,
- `MultiChoice`,
- `MultiSelect`,
- `NumericInput`,
- `Paragraph`,
- `PasswordInput`,
- `PreText`,
- `RadioButtonGroup`,
- `RadioGroup`,
- `RangeSlider`,
- `Select`,
- `Slider`,
- `Spinner`,
- `Switch`,
- `Tabs`,
- `TextAreaInput`,
- `TextInput`,
- `Toggle`.

Dodatkowo, zastosuj `HelpButton` z `Tooltip` lub dodaj `Tooltip` do przynajmniej 1 widgetu.

In [ ]:
# tutaj wpisz swoje rozwiązanie